In [1]:
import hoomd
import hoomd.md

# Save GSD trajectory

## Dumping data

HOOMD can write system configuration data to files as the simulation progresses. You can then used the saved data to visualize the system's evolution, perform off-line analysis, or as input to a follow-up simulation.

## The GSD file format

[GSD](https://bitbucket.org/glotzer/gsd) is a binary file format designed specifically for HOOMD. It supports all of HOOMD's particle properties, is easy to configure, and fast to read and write. It is trivial to use in python analysis tools, and easy to integrate with any language that can link C/C++ (only 1k lines of vanilla C code).

## Write GSD files from HOOMD

Start with the Lennard-Jones tutorial script.

In [2]:
hoomd.context.initialize('--mode=cpu');
system = hoomd.init.create_lattice(unitcell=hoomd.lattice.sc(a=2.0), n=5);
nl = hoomd.md.nlist.cell();
lj = hoomd.md.pair.lj(r_cut=3.0, nlist=nl);
lj.pair_coeff.set('A', 'A', epsilon=1.0, sigma=1.0);
all = hoomd.group.all();
hoomd.md.integrate.mode_standard(dt=0.001);
hoomd.md.integrate.langevin(group=all, kT=1.0, seed=987);

HOOMD-blue v2.1.5 CUDA (7.5) DOUBLE HPMC_MIXED MPI SSE SSE2 SSE3 SSE4_1 SSE4_2 AVX 
Compiled: 03/12/2017
Copyright 2009-2016 The Regents of the University of Michigan.
-----
You are using HOOMD-blue. Please cite the following:
* J A Anderson, C D Lorenz, and A Travesset. "General purpose molecular dynamics
  simulations fully implemented on graphics processing units", Journal of
  Computational Physics 227 (2008) 5342--5359
* J Glaser, T D Nguyen, J A Anderson, P Liu, F Spiga, J A Millan, D C Morse, and
  S C Glotzer. "Strong scaling of general-purpose molecular dynamics simulations
  on GPUs", Computer Physics Communications 192 (2015) 97--107
-----
HOOMD-blue is running on the CPU
notice(2): Group "all" created containing 125 particles
notice(2): integrate.langevin/bd is using specified gamma values


One additional command activates GSD file writes every period steps.

In [3]:
d = hoomd.dump.gsd("trajectory.gsd", period=2e3, group=all, overwrite=True);

Like all HOOMD commands, they take effect in subsequent runs:

In [4]:
hoomd.run(10000, quiet=True);

notice(2): -- Neighborlist exclusion statistics -- :
notice(2): Particles with 0 exclusions             : 125
notice(2): Neighbors included by diameter          : no
notice(2): Neighbors excluded when in the same body: no


This run produced the file `trajectory.gsd`:

In [5]:
!ls -l trajectory.gsd

-rw-rw---- 1 joaander glotzer 20227 Apr  3 20:08 trajectory.gsd


## Read a GSD file
[Full GSD tutorials are are available in the gsd docs](http://gsd.readthedocs.io/en/latest/). Here, let's just open the gsd file and verify that it has the expected number of frames and particles. We will do this with the `gsd` python module.

In [6]:
import gsd
import gsd.hoomd
t = gsd.hoomd.open('trajectory.gsd', 'rb');

In [7]:
len(t)

5

In [8]:
t[4].particles.N

125

In [9]:
t[4].configuration.step

8000

In [10]:
t[4].configuration.box

array([ 10.,  10.,  10.,   0.,   0.,   0.], dtype=float32)

In [11]:
t[4].particles.position[0:10,:]

array([[-4.1890111 , -4.70345211, -4.24545097],
       [ 1.86741066, -1.08757377,  3.26612329],
       [-3.51762629, -1.60742259,  3.95701289],
       [-1.64273536, -2.36805105,  2.59754443],
       [-3.61880612, -0.73975867,  1.48055911],
       [ 0.46584004, -4.2979784 ,  3.792485  ],
       [ 3.52424145, -1.26199448,  0.15489779],
       [ 2.24465728,  1.71063185,  3.88267493],
       [-2.30389953,  0.98935747,  0.08371102],
       [-0.42742264, -0.23506528,  1.48408175]], dtype=float32)

And here is the velocity of particle 0 at every frame in the file:

In [12]:
for f in t:
    print(f.particles.velocity[0,:])

[ 0.  0.  0.]
[ 0.  0.  0.]
[ 0.  0.  0.]
[ 0.  0.  0.]
[ 0.  0.  0.]


## Static and dynamic quantities

To save on file size, GSD does not store all particle quantities on every time step by default. By default, only particle position and orientation are written every step - and orientation is only written if it changes. This is why the previous example recorded a velocity of 0 in every frame.

Take `momentum` out of the static properties list in `dump.gsd` to store velocity in every frame output.

In [13]:
d = hoomd.dump.gsd("trajectory2.gsd", static=['attribute', 'topology'], period=2e3, group=all, overwrite=True);
hoomd.run(10000, quiet=True);

Now the velocities are saved in the file.

In [14]:
t = gsd.hoomd.open('trajectory2.gsd', 'rb');
for f in t:
    print(f.particles.velocity[0,:])

[-1.67411315 -0.57448936 -0.79786533]
[ 0.13245943  0.33422855  0.79139143]
[-0.3298862   0.22901018 -1.15823472]
[-0.00869042 -0.97660118 -2.23960042]
[-0.79439515  1.21248317  0.30553663]


For a full list of all the property groups you can specify in `static`, see the dump.gsd documentation - even the particle types, masses, charges, bond connectivity, and the number of particles/bonds/etc... can change from frame to frame.

The more properties you save on every frame, the larger the file size will be.

## Write a subset of the system

`dump.gsd` allows you to write a file that only contains a subset of the entire system.

Here is a group that contains the first 10 particles in the simulation passed to the `group` argument of `dump.gsd` file as an example.

In [15]:
first10 = hoomd.group.tag_list(name='first10', tags=[0,1,2,3,4,5,6,7,8,9]);

notice(2): Group "first10" created containing 10 particles


In [16]:
d = hoomd.dump.gsd("trajectory3.gsd", static=['attribute', 'topology'], period=2e3, group=first10, overwrite=True);
hoomd.run(10000, quiet=True);

In [17]:
t = gsd.hoomd.open('trajectory3.gsd', 'rb');

As requested, this GSD file contains only 10 particles.

In [18]:
t[4].particles.N

10

## Write a single frame GSD file

You can write a single frame GSD file with a single line. For example, use this to write the final system state at the end of a run.

In [19]:
hoomd.dump.gsd("final-frame.gsd", group=hoomd.group.all(), overwrite=True, period=None);